In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [2]:
# --- 1. VERİ YÜKLEME ---
print("Veriler yükleniyor...")
df_human = pd.read_csv("human_data_final_3000.csv")
df_ai = pd.read_csv("ai_data_generated.csv")


Veriler yükleniyor...


In [3]:
# Etiketleri sayıya çevir (Human: 0, AI: 1)
# Modelin '1' dediği şey AI olacak.
df_human['label_id'] = 0
df_ai['label_id'] = 1

# Birleştir
df = pd.concat([df_human, df_ai], ignore_index=True)
print(f"Toplam Veri: {len(df)}")

Toplam Veri: 6596


In [4]:
# --- 2. VERİ AYIRMA (GROUP SPLIT) ---
# Başlıklar (Groups) eğitim ve test setine dağılmasın diye özel ayırma yapıyoruz.
splitter = GroupShuffleSplit(test_size=0.20, n_splits=1, random_state=42)
split = splitter.split(df, groups=df['title'])
train_inds, test_inds = next(split)

train_data = df.iloc[train_inds]
test_data = df.iloc[test_inds]

X_train_raw = train_data['summary']
y_train = train_data['label_id']
X_test_raw = test_data['summary']
y_test = test_data['label_id']

print(f"Eğitim Seti: {len(train_data)} | Test Seti: {len(test_data)}")

Eğitim Seti: 5276 | Test Seti: 1320


In [5]:
# --- 3. VEKTÖRLEŞTİRME (TF-IDF) ---
# Metni matematiğe çeviriyoruz
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw) # Dikkat: Test setine sadece transform!

In [6]:
# --- 4. MODELLERİ EĞİTME VE KAYDETME ---
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, n_jobs=-1)
}

print("\n--- MODEL EĞİTİMİ VE KAYIT ---")

# Vektörleştiriciyi (matematik çeviriciyi) bir kez kaydetmemiz yeterli
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print("Vektörleştirici kaydedildi: 'tfidf_vectorizer.pkl'")

for name, model in models.items():
    # 1. Eğit
    model.fit(X_train, y_train)
    
    # 2. Test Et (Skorunu görelim)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    print(f"Model: {name}")
    print(f"Başarı Oranı: %{acc*100:.2f}")
    
    # 3. Dosya ismini oluştur ve kaydet
    # Örnek dosya adı: model_naive_bayes.pkl
    filename = f"model_{name.replace(' ', '_').lower()}.pkl"
    joblib.dump(model, filename)
    print(f"Kaydedildi: {filename}")
    print("-" * 30)

print("\nTüm modeller başarıyla kaydedildi!")


--- MODEL EĞİTİMİ VE KAYIT ---
Vektörleştirici kaydedildi: 'tfidf_vectorizer.pkl'
Model: Naive Bayes
Başarı Oranı: %94.55
Kaydedildi: model_naive_bayes.pkl
------------------------------
Model: Logistic Regression
Başarı Oranı: %97.20
Kaydedildi: model_logistic_regression.pkl
------------------------------
Model: Random Forest
Başarı Oranı: %96.82
Kaydedildi: model_random_forest.pkl
------------------------------

Tüm modeller başarıyla kaydedildi!
